In [ ]:
import os

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import skimage.io
from matplotlib import cm

%matplotlib inline
mpl.rcParams["figure.dpi"] = 300
import pandas as pd
import skimage
import skimage.io
from tqdm.notebook import tqdm

np.random.seed(0)

import os
import os.path

import h5py
from skimage.io import imread

In [ ]:
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

In [ ]:
for position in tqdm(positions):
    output_dir_np_array = f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/organoid_measurements/"
    output_dir = f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/lumen_masks_v19_05_2023/"
    stack_file = h5py.File(
        f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/denoised_registered_processed.h5",
        "r",
    )

    n_lumen = []
    lumen_vol = []
    organoid_vol = []
    stacks = np.linspace(1, 250, 250, dtype=int)
    thickness_largest_lumen = []
    for stack in stacks:
        hours = stack / 2
        if not os.path.exists(
            output_dir + f"{stack:04}" + "_lumen_organoid_mask_processed.tif"
        ):
            print(
                os.path.exists(
                    output_dir + f"{stack:04}" + "_lumen_organoid_mask_processed.tif"
                )
            )
            print(output_dir + f"{stack:04}" + "_lumen_organoid_mask_processed.tif")

In [ ]:
all_n_lumen = []
all_lumen_vol = []
all_organoid_vol = []
all_thickness_largest_lumen = []
for position in tqdm(positions):
    output_dir_np_array = f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/organoid_measurements/"
    output_dir = f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/lumen_masks_v19_05_2023/"
    stack_file = h5py.File(
        f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/denoised_registered_processed.h5",
        "r",
    )

    n_lumen = []
    lumen_vol = []
    organoid_vol = []
    stacks = np.linspace(1, 249, 125, dtype=int)
    thickness_largest_lumen = []
    for stack in tqdm(stacks):
        if not os.path.exists(
            output_dir + f"{stack:04}" + "_lumen_organoid_mask_processed.tif"
        ):
            n_lumen.append([stack / 2, np.nan])
            lumen_vol.append([stack / 2, np.nan])
            organoid_vol.append([stack / 2, np.nan])
        else:
            combined_masks = imread(
                output_dir + f"{stack:04}" + "_lumen_organoid_mask_processed.tif"
            )

            # Get lumen and organoid masks, remove lumen smaller that 20'000 mum^3
            lumen_mask = skimage.morphology.remove_small_objects(
                (combined_masks == 3), min_size=20000 / (0.347 * 0.347 * 2 * 4 * 4)
            )
            organoid_mask = (combined_masks >= 2).astype(int)
            # Number of Lumen
            label_img, num_labels = skimage.measure.label(lumen_mask, return_num=True)

            # Size of Lumen
            lumen_volume = np.count_nonzero(lumen_mask) * 0.347 * 0.347 * 2 * 4 * 4
            organoid_volume = (
                np.count_nonzero(np.asarray(organoid_mask)) * 0.347 * 0.347 * 2 * 4 * 4
            )

            n_lumen.append([stack / 2, num_labels])
            lumen_vol.append([stack / 2, lumen_volume])
            organoid_vol.append([stack / 2, organoid_volume])

    import os

    if not os.path.exists(output_dir_np_array):
        os.makedirs(output_dir_np_array)

    with open(
        output_dir_np_array + "organoid_measurements_multimosaic_v19_05_23.npy", "wb"
    ) as f:
        np.save(
            f,
            np.asarray(
                [n_lumen, lumen_vol, organoid_vol, thickness_largest_lumen],
                dtype=object,
            ),
        )

    all_n_lumen.append(n_lumen)
    all_lumen_vol.append(lumen_vol)
    all_organoid_vol.append(organoid_vol)
    all_thickness_largest_lumen.append(thickness_largest_lumen)

In [ ]:
all_n_lumen = []
all_lumen_vol = []
all_organoid_vol = []
all_thickness_largest_lumen = []
positions = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
for position in tqdm(positions):
    output_dir_np_array = f"/20210503_201032_6_lines_mosaic_HB4_D4_processed/Position_{position}_Settings_1_Processed/organoid_measurements/"
    with open(
        output_dir_np_array + "organoid_measurements_multimosaic_v19_05_23.npy", "rb"
    ) as f:
        save_numpy = np.load(f, allow_pickle=True)
    all_n_lumen.append(save_numpy[0])
    all_lumen_vol.append(save_numpy[1])
    all_organoid_vol.append(save_numpy[2])
    all_thickness_largest_lumen.append(save_numpy[3])

all_n_lumen = np.asarray(all_n_lumen)[:, ::1]
all_lumen_vol = np.asarray(all_lumen_vol)[:, ::1]
all_organoid_vol = np.asarray(all_organoid_vol)[:, ::1]
all_n_lumen[:, :, 0] = all_n_lumen[:, :, 0] - 0.5
all_lumen_vol[:, :, 0] = all_lumen_vol[:, :, 0] - 0.5
all_organoid_vol[:, :, 0] = all_organoid_vol[:, :, 0] - 0.5

# Black out position 2 after hour 50
all_n_lumen[1][50:, 1] = np.nan
all_lumen_vol[1][50:, 1] = np.nan
all_organoid_vol[1][50:, 1] = np.nan

# Add np.nans for position 4 between hours 43 and 49
all_n_lumen[3][43:49, 1] = np.nan
all_lumen_vol[3][43:49, 1] = np.nan
all_organoid_vol[3][43:49, 1] = np.nan

In [ ]:
sns.set_context("paper", rc={"font.size": 8, "axes.titlesize": 8, "axes.labelsize": 8})
rc = {
    "figure.figsize": (10, 5),
    "axes.facecolor": "white",
    "axes.grid": False,
    "grid.color": ".8",
    "font.family": "Arial",
    "font.size": 5,
}
plt.rcParams.update(rc)
cm = 1 / 2.54  # centimeters in inches

In [ ]:
lumen_df = pd.DataFrame(np.vstack(all_n_lumen), columns=["hours", "Number of lumen"])
lumen_df["days"] = 4 + (lumen_df["hours"]) / 24
lumen_mean_df = lumen_df.groupby("days").mean()

In [ ]:
lumen_mean_df.to_csv("figures_csvs/number_lumen_1_n.csv")

In [ ]:
fig, ax = plt.subplots(figsize=(8 * cm, 6 * cm))
sns.despine(left=True, bottom=True, right=True)

lumen_df = pd.DataFrame(np.vstack(all_n_lumen), columns=["hours", "Number of lumen"])
lumen_df["days"] = 4 + (lumen_df["hours"]) / 24
lumen_mean_df = lumen_df.groupby("days").mean()
sns.lineplot(x="days", y="Number of lumen", data=lumen_df, ci="sd", color="black")
print(lumen_df["days"][0])
peak_lumen = lumen_mean_df["Number of lumen"].idxmax()
plt.axvline(x=peak_lumen, color="#3d3d3d", linestyle="--")
print(peak_lumen)
"""
plt.savefig(
    f"figures/number_of_lumen_peak_at_{str(peak_lumen)}_multimosaic.svg",
    bbox_inches="tight",
)
"""

In [ ]:
fig, ax = plt.subplots(figsize=(8 * cm, 6 * cm))
sns.despine(left=True, bottom=True, right=True)
lumen_df = pd.DataFrame(
    np.vstack(all_lumen_vol),
    columns=["hours", r"Volume of imaged lumen in $\mu m^3*10^6$"],
)
lumen_df["days"] = 4 + (lumen_df["hours"]) / 24
lumen_mean_df = lumen_df.groupby("days").mean()
lumen_df[r"Volume of imaged lumen in $\mu m^3*10^6$"] = (
    lumen_df[r"Volume of imaged lumen in $\mu m^3*10^6$"] / 10**6
)

sns.lineplot(
    x="days",
    y=r"Volume of imaged lumen in $\mu m^3*10^6$",
    data=lumen_df,
    ci="sd",
    color="black",
)
print(lumen_df["days"][0])
max_vol = lumen_mean_df[r"Volume of imaged lumen in $\mu m^3*10^6$"].idxmax()
min_vol = lumen_mean_df[r"Volume of imaged lumen in $\mu m^3*10^6$"].idxmin()
plt.axvline(x=max_vol, color="#3d3d3d", linestyle="--")
plt.axvline(x=min_vol, color="#3d3d3d", linestyle="--")
print(max_vol)
print(min_vol)
"""
plt.savefig(
    f"figures/lumen_volume_multimo_max_{max_vol}_min_{min_vol}.svg", bbox_inches="tight"
)
"""

In [ ]:
fig, ax = plt.subplots(figsize=(8 * cm, 6 * cm))
sns.despine(left=True, bottom=True, right=True)
ax.set_ylim(0, 100)

# Plot the responses for different events and regions
lumen_df = pd.DataFrame(
    np.vstack(all_organoid_vol),
    columns=["hours", r"Volume of imaged organoid in $\mu m^3*10^6$"],
)
lumen_df["days"] = 4 + (lumen_df["hours"]) / 24
print(lumen_df["days"][0])
lumen_df[r"Volume of imaged organoid in $\mu m^3*10^6$"] = (
    lumen_df[r"Volume of imaged organoid in $\mu m^3*10^6$"] / 10**6
)
lumen_df = lumen_df.dropna()
lumen_df.to_csv("figures_csvs/organoid_volume_1_l.csv")
sns.lineplot(
    x="days",
    y=r"Volume of imaged organoid in $\mu m^3*10^6$",
    data=lumen_df,
    color="black",
    ci="sd",
)

plt.axvline(
    x=lumen_mean_df[r"Volume of imaged lumen in $\mu m^3*10^6$"].idxmax(),
    color="#3d3d3d",
    linestyle="--",
)
plt.axvline(
    x=lumen_mean_df[r"Volume of imaged lumen in $\mu m^3*10^6$"].idxmin(),
    color="#3d3d3d",
    linestyle="--",
)
"""
plt.savefig("figures/organoid_volume_multimosaic.svg", bbox_inches="tight")
"""

In [ ]:
all_lumen_vol_perc = all_lumen_vol.copy()
all_lumen_vol_perc[:, :, 1] = all_lumen_vol_perc[:, :, 1] / all_organoid_vol[:, :, 1]

In [ ]:
fig, ax = plt.subplots(figsize=(8 * cm, 6 * cm))
sns.despine(left=True, bottom=True, right=True)
lumen_df = pd.DataFrame(
    np.vstack(all_lumen_vol_perc), columns=["hours", r"Percent volume lumen"]
)
lumen_df["days"] = 4 + (lumen_df["hours"]) / 24
lumen_mean_df = lumen_df.groupby("days").mean()
lumen_df[r"Percent volume lumen"] = lumen_df[r"Percent volume lumen"]
sns.lineplot(x="days", y=r"Percent volume lumen", data=lumen_df, ci="sd", color="black")
print(lumen_df["days"][0])
lumen_mean_df.to_csv("figures_csvs/organoid_volume_1_m.csv")

max_vol = lumen_mean_df[r"Percent volume lumen"].idxmax()
min_vol = lumen_mean_df[r"Percent volume lumen"].idxmin()
plt.axvline(x=max_vol, color="#3d3d3d", linestyle="--")
plt.axvline(x=min_vol, color="#3d3d3d", linestyle="--")
print(max_vol)
print(min_vol)
"""
plt.savefig(
    f"figures/percent_lumen_volume_multimo_max_{max_vol}_min_{min_vol}.svg",
    bbox_inches="tight",
)
"""